# 00_preprocessing

In [1]:
import shutil
from glob import glob
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import pandas as pd
import random
from matplotlib import pyplot as plt
import json

In [4]:
!pwd

/tf


In [14]:
!ls /ainode/dataset/train/flow/

A2020-EM-01-0001_video_01  A2020-EM-01-0086_video_01  A2021-EM-01-0052_video_01
A2020-EM-01-0002_video_01  A2020-EM-01-0087_video_01  A2021-EM-01-0055_video_01
A2020-EM-01-0003_video_01  A2020-EM-01-0088_video_01  A2021-EM-01-0056_video_01
A2020-EM-01-0004_video_01  A2020-EM-01-0097_video_01  A2021-EM-01-0061_video_01
A2020-EM-01-0008_video_01  A2020-EM-01-0098_video_01  A2021-EM-01-0063_video_01
A2020-EM-01-0009_video_01  A2020-EM-01-0100_video_01  A2021-EM-01-0064_video_01
A2020-EM-01-0014_video_01  A2020-EM-01-0102_video_01  A2021-EM-01-0071_video_01
A2020-EM-01-0016_video_01  A2020-EM-01-0103_video_01  A2021-EM-01-0072_video_01
A2020-EM-01-0017_video_01  A2020-EM-01-0104_video_01  A2021-EM-01-0074_video_01
A2020-EM-01-0018_video_01  A2020-EM-01-0105_video_01  A2021-EM-01-0076_video_01
A2020-EM-01-0019_video_01  A2020-EM-01-0106_video_01  A2021-EM-01-0077_video_01
A2020-EM-01-0022_video_01  A2020-EM-01-0107_video_01  A2021-EM-01-0079_video_01
A2020-EM-01-0023_video_01  A2020-EM-01-0

In [16]:
len(glob('/ainode/dataset/train/flow/*'))

160

# Data Check

In [2]:
ANNOT_DIR = './DATA/Annotation_Train/'

In [5]:
DATA_DIR = './DATA/Train_Data/'
case_lst = [path.split('/')[-1].split('_')[0] for path in glob(DATA_DIR+'flow/*')]
case_lst.sort()

In [4]:
case_lst

[]

In [39]:
!ls /mnt/dataset/train/annotation/A2020-EM-01-0001

A2020-EM-01-0001_annotation.json


## annotation json file

In [40]:
case_idx = 0

In [43]:
case_dir = case_lst[case_idx]
case_dir

'A2020-EM-01-0001'

In [60]:
json_lst = []
for case_dir in case_lst:
    for path in glob(os.path.join(ANNOT_DIR, case_dir+'/*.json')):
        with open(path, 'r') as j:
            jsn = json.loads(j.read())
            json_lst.append(jsn)
#         print("Keys: %s" %contents.keys())

In [62]:
len(json_lst)

160

In [142]:
# fps = p_annot_df['FPS'][0]
# print(f"Frame per second: {fps}")

def frame2epoch(frameidx, fps): # image frame number (type 'int') to corresponding start epoch (type 'int')
    return int(frameidx//(30*fps)+1)

def epoch2frame(startepoch, fps): # start epoch (type 'int') to corresponding image frame start number, end number(type 'list')
    startframe = (startepoch-1)*fps*30
    return [int(startframe),int(startframe+fps*30)]

In [143]:
# 1 Epoch : 30초
# Sleep Staging: (Wake) Wake (Light Sleep) N1 N2 (Deep Sleep) N3 REM
# Tasl: 각 Epoch의 수면단게(Wake, Light Sleep, Deep Sleep) 분류

final_df = pd.DataFrame()

for case_json in json_lst:
    fps = case_json['Video_Info'][0]['Frame_Rate']
    event_df = pd.DataFrame.from_dict(case_json['Event'])
    event_df = event_df[event_df['Event_Label'].isin(['Wake', 'N1', 'N2', 'N3', 'REM'])]
    event_df['FPS'] = [fps for i in range(len(event_df))]
    event_df['Frames'] = [epoch2frame(startepoch, fps) for startepoch in event_df['Start_Epoch']]
    event_df['Case'] = [case_json['Case_Info']['Case_Number'] for i in range(len(event_df))]
    
    lst = ['Light Sleep' if value in ['N1', 'N2'] else value for value in event_df['Event_Label']]
    lst = ['Deep Sleep' if value in ['N3','REM'] else value for value in lst]
    event_df['Event_Label'] = lst
    
    final_df = pd.concat([final_df, event_df])
    
len(final_df)

132783

In [144]:
final_df

,Event_Number,Event_Label,Start_Time,End_Time,Start_Epoch,End_Epoch,Duration(second),FPS,Frames,Case
0,0,Wake,2020/05/02 22:20:30.000,2020/05/02 22:21:00.000,1,2,30.0,5.000,"[0, 150]",A2020-EM-01-0001
4,4,Wake,2020/05/02 22:21:00.000,2020/05/02 22:21:30.000,2,3,30.0,5.000,"[150, 300]",A2020-EM-01-0001
5,5,Wake,2020/05/02 22:21:30.000,2020/05/02 22:22:00.000,3,4,30.0,5.000,"[300, 450]",A2020-EM-01-0001
7,7,Wake,2020/05/02 22:22:00.000,2020/05/02 22:22:30.000,4,5,30.0,5.000,"[450, 600]",A2020-EM-01-0001
8,8,Wake,2020/05/02 22:22:30.000,2020/05/02 22:23:00.000,5,6,30.0,5.000,"[600, 750]",A2020-EM-01-0001
...,...,...,...,...,...,...,...,...,...,...
1398,1398,Light Sleep,2021/03/05 06:04:00.000,2021/03/05 06:04:30.000,838,839,30.0,4.995,"[125424, 125574]",A2021-EM-01-0170
1400,1400,Light Sleep,2021/03/05 06:04:30.000,2021/03/05 06:05:00.000,839,840,30.0,4.995,"[125574, 125724]",A2021-EM-01-0170
1403,1403,Light Sleep,2021/03/05 06:05:00.000,2021/03/05 06:05:30.000,840,841,30.0,4.995,"[125724, 125874]",A2021-EM-01-0170
1405,1405,Wake,2021/03/05 06:05:30.000,2021/03/05 06:06:00.000,841,842,30.0,4.995,"[125874, 126023]",A2021-EM-01-0170


In [146]:
# !mkdir ./DATA

final_df.to_csv(f'./DATA/annot.csv', index = False)

In [56]:
# fps = case_json['Video_Info'][0]['Frame_Rate']

In [99]:
# import pandas as pd

# event_df = pd.DataFrame.from_dict(case_json['Event'])
# # print(set(event_df['Event_Label']))
# event_df_oi = event_df[event_df['Event_Label'].isin(['Wake', 'N1', 'N2', 'N3', 'REM'])]
# event_df_oi['FPS'] = [fps for i in range(len(event_df_oi))]
# event_df_oi['Case'] = [case_json['Case_Info']['Case_Number'] for i in range(len(event_df_oi))]\

# lst = ['Light Sleep' if value in ['N1', 'N2'] else value for value in event_df_oi['Event_Label']]
# lst = ['Deep Sleep' if value in ['N3','REM'] else value for value in lst]
# event_df_oi['Event_Label'] = lst

# event_df_oi.head()
# event_df_oi['Event_Label'].value_counts()

In [58]:
# # !mkdir ./PDATA

# os.makedirs(f'./PDATA/annotation/{case_dir}/')

# event_df.to_csv(f'./PDATA/annotation/{case_dir}/{case_dir}_annot.csv', index = False)
# event_df_oi.to_csv(f'./PDATA/annotation/{case_dir}/{case_dir}_p_annot.csv', index = False)

## Images

In [125]:
# !ls /ainode/dataset/train/flow/A2020-EM-01-0001_video_01

flow_x_00000.jpg   flow_x_35425.jpg  flow_y_00000.jpg	flow_y_35425.jpg
flow_x_00001.jpg   flow_x_35426.jpg  flow_y_00001.jpg	flow_y_35426.jpg
flow_x_00002.jpg   flow_x_35427.jpg  flow_y_00002.jpg	flow_y_35427.jpg
flow_x_00003.jpg   flow_x_35428.jpg  flow_y_00003.jpg	flow_y_35428.jpg
flow_x_00004.jpg   flow_x_35429.jpg  flow_y_00004.jpg	flow_y_35429.jpg
flow_x_00005.jpg   flow_x_35430.jpg  flow_y_00005.jpg	flow_y_35430.jpg
flow_x_00006.jpg   flow_x_35431.jpg  flow_y_00006.jpg	flow_y_35431.jpg
flow_x_00007.jpg   flow_x_35432.jpg  flow_y_00007.jpg	flow_y_35432.jpg
flow_x_00008.jpg   flow_x_35433.jpg  flow_y_00008.jpg	flow_y_35433.jpg
flow_x_00009.jpg   flow_x_35434.jpg  flow_y_00009.jpg	flow_y_35434.jpg
flow_x_00010.jpg   flow_x_35435.jpg  flow_y_00010.jpg	flow_y_35435.jpg
flow_x_00011.jpg   flow_x_35436.jpg  flow_y_00011.jpg	flow_y_35436.jpg
flow_x_00012.jpg   flow_x_35437.jpg  flow_y_00012.jpg	flow_y_35437.jpg
flow_x_00013.jpg   flow_x_35438.jpg  flow_y_00013.jpg	flow_y_35438.jpg
flow_x

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [105]:
# DATA_DIR, case_dir

('/ainode/dataset/train/', 'A2021-EM-01-0170')

In [85]:
# for case_dir in 
# FLOW_DIR = f'{DATA_DIR}flow/{case_dir}_video_01'
# RGB_DIR = f'{DATA_DIR}rgb/{case_dir}_video_01'
# flow_lst = glob(FLOW_DIR+'/*.jpg')
# rgb_lst = glob(RGB_DIR+'/*.jpg')
# flow_lst.sort()
# rgb_lst.sort()

In [86]:
# fps = case_json['Video_Info'][0]['Frame_Rate']
# print(f"Frame per second: {fps}")

# def frame2epoch(frameidx): # image frame number (type 'int') to corresponding start epoch (type 'int')
#     return int(frameidx//(30*fps)+1)

# def epoch2frame(startepoch): # start epoch (type 'int') to corresponding image frame numbers (type 'list')
#     startframe = (startepoch-1)*fps*30
#     return np.arange(startframe,startframe+fps*30,1).astype('int').tolist()

Frame per second: 5.0


In [87]:
# # !rm -rf {f'./DATA/{case_dir}/flowx/'+FLOW_DIR.split('/')[-1]}
# # !rm -rf {f'./DATA/{case_dir}/flowy/'+FLOW_DIR.split('/')[-1]}

# os.makedirs(f'./DATA/flowx/{case_dir}_video_01', exist_ok=True)
# os.makedirs(f'./DATA/flowy/{case_dir}_video_01', exist_ok=True)

# flowx_lst = flow_lst[:(len(flow_lst)//2)]
# flowy_lst = flow_lst[len(flow_lst)//2:]

# for img in flowx_lst:
#     shutil.copy(img, f'./DATA/flowx/{case_dir}_video_01')
    
# for img in flowy_lst:
#     shutil.copy(img, f'./DATA/flowy/{case_dir}_video_01')

In [92]:
# os.makedirs(f'./DATA/rgb/{case_dir}_video_01', exist_ok=True)

# for img in rgb_lst:
#     shutil.copy(img, f'./DATA/rgb/{case_dir}_video_01')